In [27]:
import logging
import os
import sys

import pandas as pd

# In Jupyter, __file__ is not defined, so use the current working directory
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
# from frankenstein.graph import FrankensteinGraph
import matcher

# Suppress all logging from FrankensteinGraph and its dependencies
logging.getLogger().setLevel(logging.ERROR)

# df = pd.read_json('../eval/runs/gpt-4o-mini_answerable-full.jsonl', orient='records', lines=True)
df = pd.read_json('../eval/runs/gpt-4.1-mini_answerable-full.jsonl', orient='records', lines=True)
# df = pd.read_json('../eval/runs/Llama-3.1-8B-Instruct_answerable-full.jsonl', orient='records', lines=True)
# df = pd.read_json('../eval/runs/Qwen3-14B_answerable-full.jsonl', orient='records', lines=True)
# df = pd.read_json('../eval/runs/Llama-3.1-8B-Instruct_answerable-full.jsonl', orient='records', lines=True)


m = matcher.Matcher()

In [28]:
df.head()

,id,question_template,question,actions,answer,slot_values,answerable,data_availability,answer_format,messages,tokens,pred,correct,error
0,ca004be3-b04a-47dd-9233-3c46fb3d2db1,AverageChange,"For each year between 2010 and 2015, what was ...","[{'name': 'get_country_code_from_name', 'argum...",836871.8,"{'property': 'IS.SHP.GOOD.TU', 'subject': 'VNM...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",3428,The average yearly change in the container thr...,True,0.0
1,f4a9740a-a032-49ff-8e86-35ebd8d91dae,AverageChange,"For each year between 2005 and 2016, what was ...","[{'name': 'get_country_code_from_name', 'argum...",54.507727,"{'property': 'PA.NUS.FCRF', 'subject': 'MWI', ...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",5062,The average yearly change in the official exch...,True,0.0
2,3dd0a72d-a00e-411a-bef6-60a53dad11f6,AverageChange,"For each year between 2009 and 2022, what was ...","[{'name': 'get_country_code_from_name', 'argum...",-1.454708,"{'property': 'FM.LBL.BMNY.ZG', 'subject': 'JOR...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",5689,The average yearly change in Jordan's annual i...,True,0.0
3,1dd26f5d-f5bd-473b-8079-194da28d097a,AverageChange,"For each year between 2014 and 2018, what was ...","[{'name': 'get_country_code_from_name', 'argum...",-18345.5,"{'property': 'AG.LND.CREL.HA', 'subject': 'GBR...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",3477,The average yearly change in the land used for...,True,0.0
4,4c7f607e-2025-4033-8c42-81308dd53af4,AverageChange,What was the average yearly change in Afghanis...,"[{'name': 'get_country_code_from_name', 'argum...",-17.9,"{'property': 'AG.YLD.CREL.KG', 'subject': 'AFG...",True,full,float,"[{'role': 'system', 'content': 'You are a help...",3617,The average yearly change in Afghanistan's gra...,True,0.0


In [29]:
# Count number of nan values
df['correct'].isna().sum()

np.int64(0)

In [30]:
df['correct'].value_counts(normalize=True)

correct
True     0.7125
False    0.2875
Name: proportion, dtype: float64

In [31]:
df['correct'].groupby(df['question_template']).value_counts()

question_template           correct
AverageChange               True       18
                            False       2
AverageProperty             True       19
                            False       1
AveragePropertyComparison   True       13
                            False       7
CountryPropertyComparison   True       18
                            False       2
CountryThresholdCount       True       12
                            False       8
FactorIncreaseComparison    False      11
                            True        9
IncreasePropertyComparison  False      14
                            True        6
PropertyOfSubject           True       19
                            False       1
PropertyRatioComparison     True       20
RankChange                  True       14
                            False       6
RegionComparison            True       20
RegionComparisonResult      True       20
RegionPropertyChange        False      15
                            True        